In [1]:
!pip install faker


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import pyspark
from faker import Faker
from pyspark.sql import SparkSession, DataFrame, functions as F, types as t
from pyspark.sql.window import Window
import random
from datetime import datetime, timedelta, date

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("pyspark-test01") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()
print(spark.catalog.listDatabases())

fake = Faker()
Faker.seed(42)
random.seed(42)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 13:57:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/02 13:57:10 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/02 13:57:10 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/02 13:57:13 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/02 13:57:13 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore codespace@172.17.0.3


[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/workspaces/devcontainer-universal/spark-warehouse')]


24/04/02 13:57:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
@F.udf(t.IntegerType())
def f_int(min:int, max:int) -> int:
    return fake.random_int(min=min, max=max)

@F.udf(t.IntegerType())
def f_intUnique(digits:int) -> int:
    return fake.unique.random_number(digits=digits)

@F.udf(t.DecimalType())
def f_decimal(min:int, max:int) -> float:
    return fake.pydecimal(left_digits=12, right_digits=2, positive = True, min_value=min, max_value=max)

@F.udf(t.StringType())
def f_choise(list:list, weights:list) -> str:
    return random.choices(list,weights=weights)[0]

@F.udf(t.StringType())
def f_choise_active() -> str:
    return random.choices(['Active', 'Inactive'],weights=[0.8,0.2])[0]

@F.udf(t.BooleanType())
def f_bool() -> bool:
    return fake.boolean()

@F.udf(t.StringType())
def f_GUID() -> str:
    return fake.uuid4()

@F.udf(t.StringType())
def f_hash() -> str:
    return fake.md5(raw_output=False)

@F.udf(t.StringType())
def f_date(start:str='-3y', end:str='now') -> str:
    return str(fake.date_between(start_date=start, end_date=end))

@F.udf(t.StringType())
def f_timestamp(start:str='-3y', end:str='now') -> str:
    return str(fake.date_time_between(start_date=start, end_date=end))


In [25]:
consumers = spark.read.table('consumer_dimension').select('Consumer_Key')
consumers.count()
'''
Transaction_Header_Key	BIGINT
Division_Code_Key	BIGINT
Region_Code_Key	BIGINT
Affiliate_Code_Key	BIGINT
Market_Code_Key	BIGINT
Source_Consumer_Key	BIGINT
Cfdm_Source_System_Code_Key	BIGINT
Customer_Door_Identifier_Key	BIGINT
Local_Currency_Code_Key	BIGINT
Date_Key	INT
Transaction_Header_Cfdm_Channel_Code_Key	BIGINT
Transaction_Header_Source_Transaction_Identifier	STRING
Transaction_Header_Source_Store_Identifier	STRING
Transaction_Header_Transaction_Timestamp	TIMESTAMP
Transaction_Header_Pos_Register_Identifier	STRING
Transaction_Header_Transaction_Type_Name	STRING
Transaction_Header_Consumer_Full_Name	STRING
Transaction_Header_Consumer_Email_Address	STRING
Transaction_Header_Online_Billing_Line_1_Address	STRING
Transaction_Header_Online_Billing_Line_2_Address	STRING
Transaction_Header_Online_Billing_Line_3_Address	STRING
Transaction_Header_Online_Billing_City_Name	STRING
Transaction_Header_Online_Billing_Country_Name	STRING
Transaction_Header_Online_Billing_Postal_Code	STRING
Transaction_Header_Tourist_Order_Indicator	TINYINT
Transaction_Header_Employee_Order_Indicator	TINYINT
Transaction_Header_Guest_Order_Indicator	TINYINT
Transaction_Header_Consumer_Capture_Indicator	TINYINT
Transaction_Header_Consumer_Mac_Pro_Indicator	TINYINT
Transaction_Header_Loyalty_Transaction_Indicator	TINYINT
Transaction_Header_Total_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Saleable_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Shipping_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Discount_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Returns_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Service_Amount	DECIMAL(142)
Transaction_Header_Total_Quantity	INT
Calc_Transaction_Header_Total_Saleable_Quantity	INT
Calc_Transaction_Header_Total_Shipping_Quantity	INT
Calc_Transaction_Header_Total_Discount_Quantity	INT
Calc_Transaction_Header_Total_Return_Quantity	INT
Calc_Transaction_Header_Total_Service_Quantity	INT
Calc_Transaction_Header_Total_Sample_Quantity	INT
Calc_Transaction_Header_Total_Gift_Quantity	INT
Calc_Transaction_Header_Distinct_Sku_Count	INT
Transaction_Header_Record_Created_Timestamp	TIMESTAMP
Transaction_Header_Record_Modified_Timestamp	TIMESTAMP
Transaction_Header_Source_Consumer_Identifier	STRING
'''
tmp_df = (consumers
          .withColumn('Transaction_Header_Key', f_intUnique(F.lit(6)))  #	BIGINT
          .withColumn('Division_Code_Key', fff) #	BIGINT
          .withColumn('Region_Code_Key', fff) #	BIGINT
          .withColumn('Affiliate_Code_Key', fff)  #	BIGINT
          .withColumn('Market_Code_Key', fff) #	BIGINT
          .withColumn('Source_Consumer_Key', fff) #	BIGINT
          .withColumn('Cfdm_Source_System_Code_Key', fff) #	BIGINT
          .withColumn('Customer_Door_Identifier_Key', fff)  #	BIGINT
          .withColumn('Local_Currency_Code_Key', fff) #	BIGINT
          .withColumn('Date_Key', fff)  #	INT
          .withColumn('Transaction_Header_Cfdm_Channel_Code_Key', fff)  #	BIGINT
          .withColumn('Transaction_Header_Source_Transaction_Identifier', fff)  #	STRING
          .withColumn('Transaction_Header_Source_Store_Identifier', fff)  #	STRING
          .withColumn('Transaction_Header_Transaction_Timestamp', fff)  #	TIMESTAMP
          .withColumn('Transaction_Header_Pos_Register_Identifier', fff)  #	STRING
          .withColumn('Transaction_Header_Transaction_Type_Name', fff)  #	STRING
          .withColumn('Transaction_Header_Consumer_Full_Name', fff) #	STRING
          .withColumn('Transaction_Header_Consumer_Email_Address', fff) #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_1_Address', fff)  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_2_Address', fff)  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_3_Address', fff)  #	STRING
          .withColumn('Transaction_Header_Online_Billing_City_Name', fff) #	STRING
          .withColumn('Transaction_Header_Online_Billing_Country_Name', fff)  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Postal_Code', fff) #	STRING
          .withColumn('Transaction_Header_Tourist_Order_Indicator', fff)  #	TINYINT
          .withColumn('Transaction_Header_Employee_Order_Indicator', fff) #	TINYINT
          .withColumn('Transaction_Header_Guest_Order_Indicator', fff)  #	TINYINT
          .withColumn('Transaction_Header_Consumer_Capture_Indicator', fff) #	TINYINT
          .withColumn('Transaction_Header_Consumer_Mac_Pro_Indicator', fff) #	TINYINT
          .withColumn('Transaction_Header_Loyalty_Transaction_Indicator', fff)  #	TINYINT
          .withColumn('Transaction_Header_Total_Amount	DECIMAL', fff)  #(142)
          .withColumn('Calc_Transaction_Header_Total_Saleable_Amount	DECIMAL', fff)  #(142)
          .withColumn('Calc_Transaction_Header_Total_Shipping_Amount	DECIMAL', fff)  #(142)
          .withColumn('Calc_Transaction_Header_Total_Discount_Amount	DECIMAL', fff)  #(142)
          .withColumn('Calc_Transaction_Header_Total_Returns_Amount	DECIMAL', fff)  #(142)
          .withColumn('Calc_Transaction_Header_Total_Service_Amount	DECIMAL', fff)  #(142)
          .withColumn('Transaction_Header_Total_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Saleable_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Shipping_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Discount_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Return_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Service_Quantity', fff)  #	INT
          .withColumn('Calc_Transaction_Header_Total_Sample_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Total_Gift_Quantity', fff) #	INT
          .withColumn('Calc_Transaction_Header_Distinct_Sku_Count', fff)  #	INT
          .withColumn('Transaction_Header_Record_Created_Timestamp', fff) #	TIMESTAMP
          .withColumn('Transaction_Header_Record_Modified_Timestamp', fff)  #	TIMESTAMP
          .withColumn('Transaction_Header_Source_Consumer_Identifier', fff) #	STRING
          
          .withColumn('Consumer_Loyalty_Key', f_intUnique(F.lit(6)))
          .withColumn('Consumer_Loyalty_Program_Level_Key', f_int(F.lit(1),F.lit(10000))) # BIGINT,
          .withColumn('Consumer_Loyalty_Membership_Identifier', f_GUID()) # STRING,
          .withColumn('Consumer_Loyalty_Record_Valid_From_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Record_Valid_To_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Current_Record_Indicator', f_bool().cast('integer')) # TINYINT,
          .withColumn('Consumer_Loyalty_Anniversary_Date', f_date().cast('date')) # DATE,
          .withColumn('Consumer_Loyalty_Level_Code', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Level_Description', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Status_Code', f_choise_active()) # STRING,
          .withColumn('Consumer_Loyalty_Enrollment_Channel_Code', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Start_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Anniversary_End_Date', f_date().cast('date')) # DATE,
          .withColumn('Consumer_Loyalty_Available_Point_Number', f_int(F.lit(1),F.lit(100))) # INT,
          .withColumn('Consumer_Loyalty_Pending_Point_Number', f_int(F.lit(1),F.lit(50))) # INT,
          .withColumn('Consumer_Loyalty_Qualifying_Point_Number', f_int(F.lit(1),F.lit(50))) # INT,
          .withColumn('Consumer_Loyalty_Available_Amount', f_decimal(F.lit(1),F.lit(500))) # DECIMAL(14,2),
          .withColumn('Consumer_Loyalty_Pending_Amount', f_decimal(F.lit(1),F.lit(200))) # DECIMAL(14,2),
          .withColumn('Consumer_Loyalty_Qualifying_Amount', f_decimal(F.lit(1),F.lit(100))) # DECIMAL(14,2),
          .withColumn('Consumer_Loyalty_Last_Activity_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Active_Indicator', f_bool().cast('integer')) # TINYINT,
          .withColumn('Consumer_Loyalty_Record_Created_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Record_Modified_Timestamp', F.to_timestamp(f_timestamp())) # TIMESTAMP
)
tmp_df.show()


+------------+--------------------+----------------------------------+--------------------------------------+--------------------------------------------+------------------------------------------+-----------------------------------------+---------------------------------+---------------------------+----------------------------------+----------------------------+----------------------------------------+--------------------------------+-------------------------------------+---------------------------------------+-------------------------------------+----------------------------------------+---------------------------------+-------------------------------+----------------------------------+----------------------------------------+---------------------------------+-----------------------------------------+------------------------------------------+
|Consumer_Key|Consumer_Loyalty_Key|Consumer_Loyalty_Program_Level_Key|Consumer_Loyalty_Membership_Identifier|Consumer_Loyalty_Record_Valid_Fro

In [28]:
# spark.catalog.listTables()
# tmp_df.write.insertInto('consumer_loyalty', overwrite=True)
spark.sql('select * from consumer_loyalty').show()

+--------------------+------------+----------------------------------+--------------------------------------+--------------------------------------------+------------------------------------------+-----------------------------------------+---------------------------------+---------------------------+----------------------------------+----------------------------+----------------------------------------+--------------------------------+-------------------------------------+---------------------------------------+-------------------------------------+----------------------------------------+---------------------------------+-------------------------------+----------------------------------+----------------------------------------+---------------------------------+-----------------------------------------+------------------------------------------+
|Consumer_Loyalty_Key|Consumer_Key|Consumer_Loyalty_Program_Level_Key|Consumer_Loyalty_Membership_Identifier|Consumer_Loyalty_Record_Valid_Fro

In [36]:
groupBy_columns = ['Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key']

# Total Buyers, will join it to Consumer Dimention on Consumer Master Reg GUID
trans_head = spark.read.table('transaction_header').select('Source_Consumer_Key')
master_reg = spark.read.table('master_registry').select('Source_Consumer_Key', 'Consumer_Master_Registry_Global_Unique_Identifier')
total_buyers = (
    master_reg
    .join(trans_head, master_reg.Source_Consumer_Key == trans_head.Source_Consumer_Key, 'left')
)
master_reg.show()

+-------------------+-------------------------------------------------+
|Source_Consumer_Key|Consumer_Master_Registry_Global_Unique_Identifier|
+-------------------+-------------------------------------------------+
+-------------------+-------------------------------------------------+



In [ ]:
# Lifecycle
lifecycle = (
    spark.read.table('consumer_lifecycle')
    .select('Consumer_Key','Consumer_Lifecycle_Status_Code_Key')
    .where(F.col('Consumer_Lifecycle_Status_Current_Record_Indicator') == 1)
)

In [32]:
# Loyalty
loyalty = spark.read.table('consumer_loyalty').select('Consumer_Key','Consumer_Loyalty_Membership_Identifier').filter(F.col('Consumer_Loyalty_Active_Indicator') == 1)
loyalty.show()

+------------+--------------------------------------+
|Consumer_Key|Consumer_Loyalty_Membership_Identifier|
+------------+--------------------------------------+
|      116739|                  23b8c1e9-3924-46d...|
|       26225|                  bd9c66b3-ad3c-4d6...|
|      288389|                  17fc695a-07a0-4a6...|
|      256787|                  9a1de644-815e-46d...|
|      234053|                  b74d0fb1-32e7-462...|
|      146316|                  6b65a6a4-8b81-48f...|
|      107473|                  c241330b-01a9-471...|
|      571858|                  5be6128e-18c2-479...|
|      442417|                  ec1b8ca1-f91e-4d4...|
|       33326|                  4b0dbb41-8d52-48f...|
|       31244|                  e2acf72f-9e57-4f7...|
|       98246|                  3139d32c-93cd-49b...|
|      229258|                  a9488d99-0bbb-459...|
|      631262|                  7412b293-4729-473...|
|      588508|                  ab9099a4-35a2-40a...|
|      439898|              

In [74]:
# Contactables
phone = spark.read.table('consumer_phone').select('Consumer_Key','Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Phone_Current_Record_Indicator') == 1)
email = spark.read.table('consumer_email').select('Consumer_Key','Consumer_Email_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Email_Current_Record_Indicator') == 1)
address = spark.read.table('consumer_address').select('Consumer_Key','Consumer_Address_Postal_Opt_In_Opt_Out_Indicator','Consumer_Address_Current_Record_Indicator').where(F.col('Consumer_Address_Current_Record_Indicator')==1)
consumers = spark.read.table('consumer_dimension').select('Consumer_Key', *groupBy_columns)
contactable = (consumers
               .join(phone, consumers.Consumer_Key == phone.Consumer_Key, 'left').drop(phone.Consumer_Key)
               .join(email, consumers.Consumer_Key == email.Consumer_Key, 'left').drop(email.Consumer_Key)
               .join(address, consumers.Consumer_Key == address.Consumer_Key, 'left').drop(address.Consumer_Key)
)
contactable.show()


+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+------------------------------------------------+-----------------------------------------+
|Consumer_Key|Division_Code_Key|Region_Code_Key|Affiliate_Code_Key|Market_Code_Key|Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator|Consumer_Email_Opt_In_Opt_Out_Indicator|Consumer_Address_Postal_Opt_In_Opt_Out_Indicator|Consumer_Address_Current_Record_Indicator|
+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+------------------------------------------------+-----------------------------------------+
|        1701|                1|             10|            886963|              1|                                          0|                                   NULL|                                            NULL|       